# Scraping news from the web

In [1]:
from data.articles import load_news_articles

In [2]:
articles = load_news_articles()

Loading articles from https://www.vg.no/: 100%|██████████| 182/182 [00:39<00:00,  4.63it/s]
Loading articles from https://www.aftenposten.no/: 100%|██████████| 72/72 [00:22<00:00,  3.26it/s]
Loading articles from https://www.nettavisen.no/: 100%|██████████| 26/26 [00:25<00:00,  1.03it/s]


### Loading tokenizer

In [3]:
from data.norec.dataloader import load_full_dataset
from data.dataloader import load_data

dataset = load_full_dataset()
data = load_data(dataset)

maxlen = data['maxlen']
tokenizer = data['tokenizer']

Cleaning text...
Fitting tokenizer...
Converting texts to sequences...
Padding sequences...
One-hot encoding labels...
Done!


In [4]:
articles["text"] = articles["text"].apply(lambda x: x.lower())
articles["text"] = articles["text"].apply(lambda x: x.replace("\n", " "))
articles["text"] = articles["text"].apply(lambda x: " ".join([word for word in x.split() if word.isalpha()]))

# Predicting sentiment of news articles

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


def get_sentiment(text):
    # Loading trained model
    model = load_model('../training/models/best_model_0.807860255241394.h5')

    # Preprocessing text
    sequenced = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequenced, maxlen=maxlen)

    # Predicting sentiment
    prediction = model.predict(padded)
    sentiments = ['negative', 'neutral', 'positive']
    return sentiments[np.argmax(prediction)]


articles['sentiment'] = articles['text'].apply(get_sentiment)

In [6]:
articles

,source,url,title,text,sentiment
0,nrk,https://www.nrk.no/sport/dennis-hauger-med-for...,Dennis Hauger med første pallplass i Formel 2,dennis hauger har hatt mye stang ut i sesongen...,positive
1,aftenposten,https://www.aftenposten.no/sport/fotball/i/Wj5...,Rekdal hadde også «Molde-klausul» med HamKam: ...,rekdal hadde også med måtte være noe som var s...,neutral
2,aftenposten,https://www.aftenposten.no/norge/i/7dOgJ9/nest...,Nesten alle norske kommuner har sikret seg nok...,nesten alle norske kommuner har sikret seg nok...,neutral
3,vg,https://www.vg.no/nyheter/utenriks/i/8Q8gG1/ze...,Zelenskyj: Tror Putin vil stilles for domstol,stort volodomyr zelenskyj møtte et internasjon...,neutral
4,vg,https://www.vg.no/nyheter/meninger/i/MLr0AK/ca...,Carew-saken: Én ting er umulig å forstå,kommentar én ting er umulig å forstå av leif w...,neutral
...,...,...,...,...,...
203,vg,https://www.vg.no/nyheter/utenriks/i/34bA6L/ek...,Ekspert: − Russland ser langsiktig på konflikten,transnistria er en prorussisk landstripe øst i...,neutral
204,aftenposten,https://www.aftenposten.no/meninger/sid/i/47ew...,Det er på tide å revurdere Norges forhold til EU,det er på tide å revurdere norges forhold til ...,neutral
205,aftenposten,https://www.aftenposten.no/video/i/0GlRxM/doku...,Dokumentar: Kunsten å dø,kunsten å dø da nils arne morka foreslo å bygg...,neutral
206,aftenposten,https://www.aftenposten.no/foreldreliv/i/5GyWg...,Hvordan går det egentlig med parforholdet? Her...,hvordan går det egentlig med her kan du teste ...,positive


In [9]:
# Plot the number of sentiments
articles.sentiment.value_counts()

neutral     117
positive     90
negative      1
Name: sentiment, dtype: int64

In [21]:
print(articles[articles.sentiment == 'positive'].head(1)["text"].values)

['dennis hauger har hatt mye stang ut i sesongens fire første men etter å ha kjørt seg opp tre plasser i dagens dramatiske grand prix og opp til havnet for første gang på det var godt med en sa hauger over radioen til laget etter dobbelt på bra sa haugers lagkamerat jehan daruvala etter å ha kjørt over målstreken til en men det var ikke uten dramatikk at hauger tok sin første for mens førerne kjørte i farten til den virtuelle sikkerhetsbilen i den så det ikke ut til at dennis hauger fulgte med i hva i all verden skjedde dennis er avhengig av informasjon i det dette tapte han masse sa viaplays ekspertkommentator henning isdal etter dennis hauger på et pressetreff på operaen i annika byrde ntb han sløvet da lagkamerat jehan daruvala ga gass på riktig hang hauger langt etter og da tidene dukket opp hadde hauger tapt hele to der følger ikke dennis han sløvet rett og slett og tapte to sekunder på sa henning rune bendiksen viaplay rett etter den trege omstarten viste lagsjefene i prema som h

In [22]:
print(articles[articles.sentiment == 'neutral'].head(1)["text"].values)

['rekdal hadde også med måtte være noe som var større trondheim han kunne blitt allerede i og kjetil rekdal innser at før desember i fjor ville oddsen vært lavere på at han heller skulle blitt sjef på aker stadion enn selv hadde han sikret seg i begge med kjetil rekdal i diskusjon med dommer svein oddvar moen etter i sarpsborg foran erlend dahl i dag aftenposten samarbeider med derfor får du som også artikler og tjenester som er laget av i hans tidligere sto nemlig fire navngitte norske klubber som skulle gi ham en fra der var rosenborg og molde samt brann og vålerenga og alle muligheter fra det bekrefter rekdal overfor det er og hvis du tenker du bryter ikke en avtale med hamkam eller bruker klausulen til en klubb på samme det måtte være noe som var mye begrunner søndag skal han for første gang se på rivaloppgjøret mellom rosenborg og hans første seniorklubb en år gammel rekdal var faktisk sentral da uvennskapet oppsto i løpet av noen intense oppgjør i serie og cup i han scoret nemlig

In [23]:
print(articles[articles.sentiment == 'negative'].head(1)["text"].values)

['nyter vilde hall soler seg foran nyt sommeren før været typisk med tilbakeslag mens helgen byr på sommervær i kommer kong vinter luskende fra nord neste enda mer grunn til å nyte solen når den først er sier vilde av sven arne buggeland oppdatert i går med oppbrettet og solbriller på nesen gleder seg over godværet på eidsvolls plass foran før hun skal møte venninner fra det er veldig sier til men vakthavende meteorolog trond lien minner om at det ikke er fint vær det er regn og sludd i troms og sier lien ved meteorologisk mens han gikk på jobb i tromsø i fire grader er det over grader sør på østlandet og enkelte steder i grunnen er været fint fra nordland og så marie solstrand og harald otterå gjorde det eneste på marie solstrand og harald otterå fra harstad er på småbarnsforeldrene satte seg på fra harstad i regn og surt vær fredag hun med ulltøy og han med kortermet sommerskjorte og bare jeg skulle jo til sier han hadde lest at oslo var varmest i europa det er jo sykt varmt det var 